셀 1: 라이브러리 및 기본 클래스 정의

In [1]:
# ------------------------------------------------------
# Cognita 에이전트 - 관계형 데이터 기반 위험도 분석 시스템
# 셀 1: 기본 설정 및 데이터 클래스
# ------------------------------------------------------

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from neo4j import GraphDatabase
import logging
from dataclasses import dataclass
from collections import defaultdict
from typing import Dict, List, Tuple, Optional
import json

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class RiskMetrics:
    """위험 지표 데이터 클래스"""
    employee_id: str
    manager_instability_score: float
    team_volatility_index: float
    social_isolation_index: float
    network_centrality_score: float
    overall_risk_score: float
    risk_category: str
    risk_factors: List[str]
    network_stats: Dict[str, float]

print("✅ Cognita 관계형 위험도 분석 시스템 - 기본 클래스 정의 완료")

✅ Cognita 관계형 위험도 분석 시스템 - 기본 클래스 정의 완료


셀 2: CognitaRiskAnalyzer 기본 클래스

In [2]:
# ------------------------------------------------------
# 셀 2: CognitaRiskAnalyzer 핵심 클래스 정의
# ------------------------------------------------------

class CognitaRiskAnalyzer:
    """Cognita 에이전트 관계형 위험도 분석기"""
    
    def __init__(self, neo4j_manager):
        self.neo4j_manager = neo4j_manager
        self.analysis_date = datetime.now()
        
    def analyze_employee_risk(self, employee_id: str) -> RiskMetrics:
        """특정 직원의 종합 위험도 분석"""
        
        logger.info(f"직원 {employee_id} 위험도 분석 시작")
        
        # 1. 관리자 안정성 분석
        manager_score = self.analyze_manager_stability(employee_id)
        
        # 2. 팀 변동성 분석  
        team_volatility = self.analyze_team_volatility(employee_id)
        
        # 3. 사회적 네트워크 중심성 분석
        centrality_score, network_stats = self.analyze_network_centrality(employee_id)
        
        # 4. 사회적 고립 지수 계산
        isolation_score = self.calculate_social_isolation(employee_id)
        
        # 5. 종합 위험 점수 계산
        overall_score = self.calculate_overall_risk_score(
            manager_score, team_volatility, centrality_score, isolation_score
        )
        
        # 6. 위험 범주 및 주요 요인 결정
        risk_category, risk_factors = self.determine_risk_category(
            manager_score, team_volatility, centrality_score, isolation_score
        )
        
        return RiskMetrics(
            employee_id=employee_id,
            manager_instability_score=manager_score,
            team_volatility_index=team_volatility,
            social_isolation_index=isolation_score,
            network_centrality_score=centrality_score,
            overall_risk_score=overall_score,
            risk_category=risk_category,
            risk_factors=risk_factors,
            network_stats=network_stats
        )

print("✅ CognitaRiskAnalyzer 클래스 정의 완료")

# ------------------------------------------------------
# 셀 2-1: 성능 최적화를 위한 추가 인덱스 생성 (안전 처리)
# ------------------------------------------------------

def create_performance_indexes(neo4j_manager):
    """분석 성능 향상을 위한 추가 인덱스 생성"""
    
    print("성능 최적화 인덱스 생성 중...")
    
    optimization_queries = [
        # 협업 관계 최적화 인덱스
        "CREATE INDEX IF NOT EXISTS FOR ()-[r:COLLABORATES_WITH]-() ON (r.collaboration_strength)",
        "CREATE INDEX IF NOT EXISTS FOR ()-[r:COLLABORATES_WITH]-() ON (r.interaction_frequency)",
        "CREATE INDEX IF NOT EXISTS FOR ()-[r:COLLABORATES_WITH]-() ON (r.relationship_quality)",
        
        # 프로젝트 관계 최적화
        "CREATE INDEX IF NOT EXISTS FOR ()-[r:PARTICIPATES_IN]-() ON (r.role_in_project)",
        "CREATE INDEX IF NOT EXISTS FOR (p:Project) ON (p.status)",
        
        # 복합 인덱스 (가능한 경우)
        "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.department, e.job_level)",
    ]
    
    with neo4j_manager.driver.session() as session:
        for query in optimization_queries:
            try:
                session.run(query)
                print(f"  ✓ 생성: {query.split('ON')[1] if 'ON' in query else 'constraint'}")
            except Exception as e:
                print(f"  ✗ 실패: {str(e)}")

# Neo4j 연결 확인 후 인덱스 생성
try:
    # neo4j_manager가 정의되어 있는지 확인
    if 'neo4j_manager' in globals() and neo4j_manager:
        create_performance_indexes(neo4j_manager)
        print("인덱스 생성 완료")
    else:
        print("⚠️ neo4j_manager가 정의되지 않았습니다.")
        print("먼저 Neo4j 연결을 설정하세요:")
        print("1. 셀 10(Neo4j 연결 설정)을 먼저 실행하거나")
        print("2. 아래 코드를 실행하여 연결하세요:")
        print()
        print("# Neo4j 연결 설정 예시:")
        print("# from neo4j import GraphDatabase")
        print("# neo4j_manager = Neo4jManager('bolt://...', 'username', 'password')")
        
except NameError:
    print("⚠️ neo4j_manager 변수가 존재하지 않습니다.")
    print("Neo4j 연결을 먼저 설정해주세요.")
    print()
    print("다음 순서로 진행하세요:")
    print("1. Neo4j 연결 설정 코드 실행")
    print("2. 이 셀 다시 실행")

# ------------------------------------------------------
# 셀 2-2: Neo4j 연결 간편 설정 (필요시)
# ------------------------------------------------------

import time
from neo4j import GraphDatabase

# Neo4j 연결 설정 (이전 성공한 설정 사용)
NEO4J_CONFIG = {
    "uri": "bolt://54.162.43.24:7687",
    "username": "neo4j",
    "password": "resident-success-moss"
}

class Neo4jManager:
    """Neo4j 데이터베이스 연결 및 관리 클래스"""
    
    def __init__(self, uri, username, password, max_retries=3):
        self.uri = uri
        self.username = username
        self.password = password
        self.max_retries = max_retries
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Neo4j 연결 설정"""
        for attempt in range(self.max_retries):
            try:
                print(f"Neo4j 연결 시도 {attempt + 1}/{self.max_retries}: {self.uri}")
                
                self.driver = GraphDatabase.driver(
                    self.uri, 
                    auth=(self.username, self.password),
                    max_connection_lifetime=30,
                    max_connection_pool_size=10,
                    connection_acquisition_timeout=30,
                    connection_timeout=10
                )
                
                # 연결 테스트
                with self.driver.session() as session:
                    result = session.run("RETURN 'Connected' as status")
                    print("Neo4j 연결 성공!")
                    return
                    
            except Exception as e:
                print(f"연결 시도 {attempt + 1} 실패: {str(e)}")
                
                if self.driver:
                    try:
                        self.driver.close()
                    except:
                        pass
                    self.driver = None
                
                if attempt < self.max_retries - 1:
                    wait_time = (attempt + 1) * 2
                    print(f"{wait_time}초 후 재시도...")
                    time.sleep(wait_time)
                else:
                    raise ConnectionError(f"Neo4j 연결 실패: {str(e)}")
    
    def close(self):
        if self.driver:
            try:
                self.driver.close()
                print("Neo4j 연결 종료")
            except Exception as e:
                print(f"연결 종료 중 오류: {str(e)}")
        self.driver = None

# neo4j_manager가 없는 경우에만 생성
if 'neo4j_manager' not in globals() or neo4j_manager is None:
    try:
        print("Neo4j 연결 생성 중...")
        neo4j_manager = Neo4jManager(
            NEO4J_CONFIG['uri'], 
            NEO4J_CONFIG['username'], 
            NEO4J_CONFIG['password']
        )
        
        # 데이터 확인
        with neo4j_manager.driver.session() as session:
            result = session.run("MATCH (e:Employee) RETURN count(e) as count")
            employee_count = result.single()["count"]
            print(f"연결 확인: 직원 {employee_count:,}명")
        
    except Exception as e:
        print(f"Neo4j 연결 실패: {str(e)}")
        neo4j_manager = None
else:
    print("Neo4j 연결이 이미 설정되어 있습니다.")

✅ CognitaRiskAnalyzer 클래스 정의 완료
⚠️ neo4j_manager가 정의되지 않았습니다.
먼저 Neo4j 연결을 설정하세요:
1. 셀 10(Neo4j 연결 설정)을 먼저 실행하거나
2. 아래 코드를 실행하여 연결하세요:

# Neo4j 연결 설정 예시:
# from neo4j import GraphDatabase
# neo4j_manager = Neo4jManager('bolt://...', 'username', 'password')
Neo4j 연결 생성 중...
Neo4j 연결 시도 1/3: bolt://54.162.43.24:7687
Neo4j 연결 성공!
연결 확인: 직원 1,470명


셀 3: 관리자 안정성 분석

In [3]:
# ------------------------------------------------------
# 셀 3-최적화: 관리자 안정성 분석 (성능 최적화)
# ------------------------------------------------------

def analyze_manager_stability_optimized(self, employee_id: str) -> float:
    """관리자 안정성 분석 - 최적화된 버전"""
    
    # 단순화된 쿼리로 성능 향상
    query = """
    MATCH (emp:Employee {employee_id: $employee_id})
    
    // 직속 상사만 조회 (복잡한 부서 분석 제거)
    OPTIONAL MATCH (emp)-[reports:REPORTS_TO]->(manager:Employee)
    
    // 관리자 부하 직원 수 (서브쿼리 단순화)
    OPTIONAL MATCH (manager)<-[:REPORTS_TO]-(subordinate:Employee)
    
    RETURN 
        manager.employee_id as direct_manager,
        reports.reporting_frequency as reporting_frequency,
        count(subordinate) as manager_load
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, employee_id=employee_id)
        record = result.single()
        
        if not record:
            return 0.7  # 기본 불안정성 점수
        
        manager_load = record.get('manager_load', 0)
        reporting_frequency = record.get('reporting_frequency', 'rarely')
        
        instability_score = 0.0
        
        # 직속 관리자 부재
        if not record.get('direct_manager'):
            instability_score += 0.5
        
        # 관리자 과부하 (단순화된 계산)
        if manager_load > 12:
            instability_score += 0.3
        
        # 보고 빈도 (단순화된 매핑)
        frequency_penalty = {'daily': 0.0, 'weekly': 0.1, 'monthly': 0.2, 'rarely': 0.3}
        instability_score += frequency_penalty.get(reporting_frequency, 0.3)
        
        return min(instability_score, 1.0)

# 기존 메서드 교체
CognitaRiskAnalyzer.analyze_manager_stability = analyze_manager_stability_optimized

print("✅ 관리자 안정성 분석 최적화 완료")

✅ 관리자 안정성 분석 최적화 완료


셀 4: 사회적 네트워크 중심성 분석

In [4]:
# ------------------------------------------------------
# 셀 4-최적화: 네트워크 중심성 분석 (샘플링 최적화)
# ------------------------------------------------------

def analyze_network_centrality_optimized(self, employee_id: str) -> Tuple[float, Dict[str, float]]:
    """네트워크 중심성 분석 - 샘플링으로 최적화"""
    
    # 협업 관계 샘플링 (성능 향상)
    query = """
    MATCH (emp:Employee {employee_id: $employee_id})
    
    // 직접 협업 관계만 조회 (2-hop 제거로 성능 향상)
    OPTIONAL MATCH (emp)-[collab:COLLABORATES_WITH]-(colleague:Employee)
    WHERE collab.collaboration_strength IS NOT NULL
    
    // 상위 협업 관계만 샘플링 (강도 기준)
    WITH emp, collab, colleague
    ORDER BY collab.collaboration_strength DESC
    LIMIT 50  // 최대 50개 관계만 분석
    
    // 프로젝트 연결 (현재 활성 프로젝트만)
    OPTIONAL MATCH (emp)-[:PARTICIPATES_IN]->(proj:Project {status: 'active'})<-[:PARTICIPATES_IN]-(proj_colleague:Employee)
    WHERE proj_colleague.employee_id <> emp.employee_id
    
    RETURN 
        count(DISTINCT colleague.employee_id) as direct_connections,
        avg(collab.collaboration_strength) as avg_collaboration_strength,
        count(DISTINCT proj_colleague.employee_id) as project_connections,
        
        // 관계 품질 (간단화)
        sum(CASE WHEN collab.relationship_quality = 'excellent' THEN 1 ELSE 0 END) as excellent_count,
        sum(CASE WHEN collab.relationship_quality = 'poor' THEN 1 ELSE 0 END) as poor_count,
        
        // 빈번한 상호작용
        sum(CASE WHEN collab.interaction_frequency IN ['daily', 'weekly'] THEN 1 ELSE 0 END) as frequent_count
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, employee_id=employee_id)
        record = result.single()
        
        if not record:
            return 0.0, {}
        
        direct_connections = record.get('direct_connections', 0)
        avg_strength = record.get('avg_collaboration_strength', 0.0) or 0.0
        project_connections = record.get('project_connections', 0)
        excellent_count = record.get('excellent_count', 0)
        poor_count = record.get('poor_count', 0)
        frequent_count = record.get('frequent_count', 0)
        
        # 단순화된 중심성 계산
        degree_centrality = min(direct_connections / 15.0, 1.0)  # 15명 기준
        strength_centrality = min(avg_strength, 1.0)
        project_centrality = min(project_connections / 8.0, 1.0)  # 8명 기준
        
        # 관계 품질 점수
        total_relations = direct_connections
        quality_score = 0.5  # 기본값
        if total_relations > 0:
            quality_score = (excellent_count * 1.0 + (total_relations - excellent_count - poor_count) * 0.5) / total_relations
        
        # 상호작용 점수
        interaction_score = min(frequent_count / 5.0, 1.0)  # 5명 기준
        
        # 가중 평균 (단순화)
        centrality_score = (
            degree_centrality * 0.4 +
            strength_centrality * 0.3 +
            project_centrality * 0.2 +
            interaction_score * 0.1
        )
        
        network_stats = {
            'degree_centrality': degree_centrality,
            'strength_centrality': strength_centrality,
            'project_centrality': project_centrality,
            'quality_score': quality_score,
            'direct_connections': direct_connections,
            'avg_strength': avg_strength
        }
        
        return centrality_score, network_stats

# 메서드 교체
CognitaRiskAnalyzer.analyze_network_centrality = analyze_network_centrality_optimized

print("✅ 네트워크 중심성 분석 최적화 완료 (샘플링 적용)")

✅ 네트워크 중심성 분석 최적화 완료 (샘플링 적용)


셀 5: 사회적 고립 지수 계산

In [5]:
# ------------------------------------------------------
# 셀 5-최적화: 사회적 고립 지수 계산 (경량화)
# ------------------------------------------------------

def calculate_social_isolation_optimized(self, employee_id: str) -> float:
    """사회적 고립 지수 계산 - 경량화된 버전"""
    
    # 핵심 지표만 조회하는 단순화된 쿼리
    query = """
    MATCH (emp:Employee {employee_id: $employee_id})
    
    // Layer 1: 직접 협업 (상위 관계만)
    OPTIONAL MATCH (emp)-[collab:COLLABORATES_WITH]-(colleague:Employee)
    WHERE collab.collaboration_strength >= 0.3  // 의미있는 협업만
    
    // Layer 2: 현재 활성 프로젝트만
    OPTIONAL MATCH (emp)-[:PARTICIPATES_IN]->(proj:Project {status: 'active'})<-[:PARTICIPATES_IN]-(proj_colleague:Employee)
    WHERE proj_colleague.employee_id <> emp.employee_id
    
    // Layer 3: 관리 관계
    OPTIONAL MATCH (emp)-[:REPORTS_TO]->(manager:Employee)
    OPTIONAL MATCH (subordinate:Employee)-[:REPORTS_TO]->(emp)
    
    RETURN 
        count(DISTINCT colleague.employee_id) as meaningful_collaborations,
        count(DISTINCT proj_colleague.employee_id) as active_project_colleagues,
        count(DISTINCT manager.employee_id) as has_manager,
        count(DISTINCT subordinate.employee_id) as subordinates,
        
        avg(collab.collaboration_strength) as avg_strength,
        sum(CASE WHEN collab.interaction_frequency IN ['daily', 'weekly'] THEN 1 ELSE 0 END) as frequent_interactions
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, employee_id=employee_id)
        record = result.single()
        
        if not record:
            return 1.0  # 완전 고립
        
        meaningful_collaborations = record.get('meaningful_collaborations', 0)
        active_project_colleagues = record.get('active_project_colleagues', 0)
        has_manager = record.get('has_manager', 0)
        subordinates = record.get('subordinates', 0)
        avg_strength = record.get('avg_strength', 0.0) or 0.0
        frequent_interactions = record.get('frequent_interactions', 0)
        
        # 단순화된 고립 점수 계산
        isolation_score = 0.0
        
        # 핵심 고립 요인들
        if meaningful_collaborations == 0:
            isolation_score += 0.4
        elif meaningful_collaborations < 2:
            isolation_score += 0.2
        
        if active_project_colleagues == 0:
            isolation_score += 0.3
        
        if has_manager == 0 and subordinates == 0:
            isolation_score += 0.2
        
        if avg_strength < 0.4:
            isolation_score += 0.2
        
        if frequent_interactions == 0 and meaningful_collaborations > 0:
            isolation_score += 0.1
        
        return min(isolation_score, 1.0)

# 메서드 교체
CognitaRiskAnalyzer.calculate_social_isolation = calculate_social_isolation_optimized

print("✅ 사회적 고립 지수 계산 최적화 완료 (경량화)")

✅ 사회적 고립 지수 계산 최적화 완료 (경량화)


셀 6: 팀 변동성 분석

In [6]:
# ------------------------------------------------------
# 셀 6-최적화: 팀 변동성 분석 (단순화)
# ------------------------------------------------------

def analyze_team_volatility_optimized(self, employee_id: str) -> float:
    """팀 변동성 분석 - 단순화된 버전"""
    
    # 핵심 팀 정보만 조회
    query = """
    MATCH (emp:Employee {employee_id: $employee_id})-[:WORKS_IN]->(dept:Department)
    
    // 부서 팀원 수
    MATCH (dept_teammate:Employee)-[:WORKS_IN]->(dept)
    
    // 현재 활성 프로젝트 수만
    OPTIONAL MATCH (emp)-[:PARTICIPATES_IN]->(proj:Project {status: 'active'})
    
    RETURN 
        dept.name as department,
        count(DISTINCT dept_teammate.employee_id) - 1 as dept_team_size,  // -1 for self
        count(DISTINCT proj.project_id) as active_projects
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, employee_id=employee_id)
        record = result.single()
        
        if not record:
            return 0.5
        
        dept_team_size = record.get('dept_team_size', 0)
        active_projects = record.get('active_projects', 0)
        
        volatility_score = 0.0
        
        # 단순화된 변동성 계산
        if dept_team_size < 3:
            volatility_score += 0.4  # 매우 작은 팀
        elif dept_team_size < 8:
            volatility_score += 0.2  # 작은 팀
        
        if active_projects == 0:
            volatility_score += 0.3  # 프로젝트 미참여
        elif active_projects > 5:
            volatility_score += 0.2  # 너무 많은 프로젝트
        
        return min(volatility_score, 1.0)

# 메서드 교체
CognitaRiskAnalyzer.analyze_team_volatility = analyze_team_volatility_optimized

print("✅ 팀 변동성 분석 최적화 완료 (단순화)")

✅ 팀 변동성 분석 최적화 완료 (단순화)


셀7: 종합 위험 점수 계산

In [7]:
# ------------------------------------------------------
# 셀 7: 종합 위험 점수 계산 및 위험 범주 결정
# ------------------------------------------------------

def calculate_overall_risk_score(self, manager_score: float, team_volatility: float,
                               centrality_score: float, isolation_score: float) -> float:
    """종합 위험 점수 계산 (가중 평균)"""
    
    # 가중치 설정 (관계형 분석에 특화된 중요도)
    weights = {
        'social_isolation': 0.35,        # 사회적 고립 (가장 중요)
        'low_centrality': 0.25,          # 낮은 네트워크 중심성
        'manager_instability': 0.25,     # 관리자 불안정성
        'team_volatility': 0.15          # 팀 변동성
    }
    
    # 중심성은 역수로 변환 (낮을수록 위험)
    low_centrality_score = 1.0 - centrality_score
    
    overall_score = (
        isolation_score * weights['social_isolation'] +
        low_centrality_score * weights['low_centrality'] +
        manager_score * weights['manager_instability'] +
        team_volatility * weights['team_volatility']
    )
    
    return min(overall_score, 1.0)

def determine_risk_category(self, manager_score: float, team_volatility: float,
                           centrality_score: float, isolation_score: float) -> Tuple[str, List[str]]:
    """위험 범주 및 주요 요인 결정"""
    
    overall_score = self.calculate_overall_risk_score(
        manager_score, team_volatility, centrality_score, isolation_score
    )
    
    # 위험 범주 결정
    if overall_score >= 0.7:
        risk_category = "HIGH"
    elif overall_score >= 0.4:
        risk_category = "MEDIUM"
    else:
        risk_category = "LOW"
    
    # 주요 위험 요인 식별 (임계값 기반)
    risk_factors = []
    
    if isolation_score > 0.6:
        risk_factors.append("사회적_고립")
    if centrality_score < 0.3:
        risk_factors.append("네트워크_중심성_저하")
    if manager_score > 0.5:
        risk_factors.append("관리자_불안정성")
    if team_volatility > 0.5:
        risk_factors.append("팀_변동성")
    
    # 추가적인 세부 요인
    if isolation_score > 0.4 and centrality_score < 0.4:
        risk_factors.append("복합_네트워크_약화")
    if manager_score > 0.3 and team_volatility > 0.3:
        risk_factors.append("구조적_불안정성")
    
    return risk_category, risk_factors

# CognitaRiskAnalyzer 클래스에 메서드 추가
CognitaRiskAnalyzer.calculate_overall_risk_score = calculate_overall_risk_score
CognitaRiskAnalyzer.determine_risk_category = determine_risk_category

print("✅ 종합 위험 점수 계산 및 분류 함수 추가 완료")

✅ 종합 위험 점수 계산 및 분류 함수 추가 완료


셀 8: 일괄 분석 및 보고서 생성

In [8]:
# ------------------------------------------------------
# 셀 8-최적화: 배치 분석 최적화 (병렬 처리 시뮬레이션)
# ------------------------------------------------------

def batch_analyze_department_optimized(self, department_name: str, sample_size: int = 20) -> List[RiskMetrics]:
    """부서 분석 - 샘플링과 배치 최적화"""
    
    # 샘플링된 직원 목록 조회
    query = """
    MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department {name: $dept_name})
    WITH emp
    ORDER BY emp.employee_id
    LIMIT $sample_size
    RETURN collect({
        employee_id: emp.employee_id,
        name: emp.name
    }) as employees
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, dept_name=department_name, sample_size=sample_size)
        record = result.single()
        
        if not record:
            return []
        
        employees = record.get('employees', [])
    
    print(f"부서 '{department_name}' 샘플 분석: {len(employees)}명")
    
    risk_analyses = []
    batch_size = 5  # 작은 배치로 처리
    
    for i in range(0, len(employees), batch_size):
        batch = employees[i:i+batch_size]
        
        print(f"  배치 {i//batch_size + 1}: {len(batch)}명 처리 중...")
        
        for emp_info in batch:
            emp_id = emp_info['employee_id']
            try:
                risk_metrics = self.analyze_employee_risk(emp_id)
                risk_analyses.append(risk_metrics)
            except Exception as e:
                logger.warning(f"직원 {emp_id} 분석 실패: {str(e)}")
                continue
    
    return risk_analyses

# 메서드 교체
CognitaRiskAnalyzer.batch_analyze_department = batch_analyze_department_optimized

print("✅ 배치 분석 최적화 완료 (샘플링 + 배치 처리)")

✅ 배치 분석 최적화 완료 (샘플링 + 배치 처리)


In [15]:
# ------------------------------------------------------
# 누락된 메서드들 정의 및 클래스에 추가
# ------------------------------------------------------

def generate_risk_report(self, risk_metrics: List[RiskMetrics]) -> Dict:
    """위험도 분석 종합 보고서 생성"""
    
    if not risk_metrics:
        return {"error": "분석 데이터 없음"}
    
    # 위험도별 분류
    high_risk = [r for r in risk_metrics if r.risk_category == "HIGH"]
    medium_risk = [r for r in risk_metrics if r.risk_category == "MEDIUM"]
    low_risk = [r for r in risk_metrics if r.risk_category == "LOW"]
    
    # 주요 위험 요인 통계
    all_factors = [factor for r in risk_metrics for factor in r.risk_factors]
    factor_counts = defaultdict(int)
    for factor in all_factors:
        factor_counts[factor] += 1
    
    # 평균 점수 계산
    avg_scores = {
        'overall_risk': np.mean([r.overall_risk_score for r in risk_metrics]),
        'social_isolation': np.mean([r.social_isolation_index for r in risk_metrics]),
        'network_centrality': np.mean([r.network_centrality_score for r in risk_metrics]),
        'manager_instability': np.mean([r.manager_instability_score for r in risk_metrics]),
        'team_volatility': np.mean([r.team_volatility_index for r in risk_metrics])
    }
    
    # 네트워크 통계
    network_stats = defaultdict(list)
    for r in risk_metrics:
        if r.network_stats:
            for key, value in r.network_stats.items():
                if isinstance(value, (int, float)):
                    network_stats[key].append(value)
    
    avg_network_stats = {key: np.mean(values) for key, values in network_stats.items() if values}
    
    # 보고서 구성
    report = {
        "분석_개요": {
            "총_분석_인원": len(risk_metrics),
            "고위험_인원": len(high_risk),
            "중위험_인원": len(medium_risk),
            "저위험_인원": len(low_risk),
            "고위험_비율": f"{len(high_risk)/len(risk_metrics)*100:.1f}%",
            "분석_일시": self.analysis_date.isoformat()
        },
        "위험_분포": {
            "HIGH": len(high_risk),
            "MEDIUM": len(medium_risk), 
            "LOW": len(low_risk)
        },
        "주요_위험_요인_빈도": dict(sorted(factor_counts.items(), key=lambda x: x[1], reverse=True)),
        "평균_위험_점수": {key: round(value, 3) for key, value in avg_scores.items()},
        "네트워크_통계": {key: round(value, 3) for key, value in avg_network_stats.items()},
        "고위험_직원_상세": [
            {
                "employee_id": r.employee_id,
                "overall_risk_score": round(r.overall_risk_score, 3),
                "social_isolation": round(r.social_isolation_index, 3),
                "network_centrality": round(r.network_centrality_score, 3),
                "primary_risk_factors": r.risk_factors[:3]
            }
            for r in sorted(high_risk, key=lambda x: x.overall_risk_score, reverse=True)
        ],
        "권장_조치사항": self._generate_recommendations(risk_metrics)
    }
    
    return report

def _generate_recommendations(self, risk_metrics: List[RiskMetrics]) -> List[str]:
    """위험도 분석 결과 기반 권장 조치사항 생성"""
    
    recommendations = []
    
    # 고위험 직원 수 기반
    high_risk_count = len([r for r in risk_metrics if r.risk_category == "HIGH"])
    total_count = len(risk_metrics)
    
    if high_risk_count > total_count * 0.2:  # 20% 이상 고위험
        recommendations.append("부서 전체 팀 빌딩 프로그램 시급 실시")
        recommendations.append("관리 구조 재점검 및 개선")
    
    # 주요 위험 요인별 권장사항
    all_factors = [factor for r in risk_metrics for factor in r.risk_factors]
    factor_counts = defaultdict(int)
    for factor in all_factors:
        factor_counts[factor] += 1
    
    if factor_counts.get("사회적_고립", 0) > total_count * 0.3:
        recommendations.append("멘토링 프로그램 확대 운영")
        recommendations.append("정기 1:1 미팅 및 소통 강화")
    
    if factor_counts.get("네트워크_중심성_저하", 0) > total_count * 0.3:
        recommendations.append("크로스 기능 팀 프로젝트 증진")
        recommendations.append("사내 네트워킹 이벤트 활성화")
    
    if factor_counts.get("관리자_불안정성", 0) > total_count * 0.2:
        recommendations.append("관리자 교육 및 코칭 강화")
        recommendations.append("관리 스팬 최적화 검토")
    
    return recommendations if recommendations else ["현재 위험 수준 양호, 지속 모니터링 권장"]

def batch_analyze_department_optimized(self, department_name: str, sample_size: int = 20) -> List[RiskMetrics]:
    """부서 분석 - 샘플링과 배치 최적화"""
    
    # 샘플링된 직원 목록 조회
    query = """
    MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department {name: $dept_name})
    WITH emp
    ORDER BY emp.employee_id
    LIMIT $sample_size
    RETURN collect({
        employee_id: emp.employee_id,
        name: emp.name
    }) as employees
    """
    
    with self.neo4j_manager.driver.session() as session:
        result = session.run(query, dept_name=department_name, sample_size=sample_size)
        record = result.single()
        
        if not record:
            return []
        
        employees = record.get('employees', [])
    
    print(f"부서 '{department_name}' 샘플 분석: {len(employees)}명")
    
    risk_analyses = []
    batch_size = 5  # 작은 배치로 처리
    
    for i in range(0, len(employees), batch_size):
        batch = employees[i:i+batch_size]
        
        print(f"  배치 {i//batch_size + 1}: {len(batch)}명 처리 중...")
        
        for emp_info in batch:
            emp_id = emp_info['employee_id']
            try:
                risk_metrics = self.analyze_employee_risk(emp_id)
                risk_analyses.append(risk_metrics)
            except Exception as e:
                logger.warning(f"직원 {emp_id} 분석 실패: {str(e)}")
                continue
    
    return risk_analyses

# 누락된 메서드들을 CognitaRiskAnalyzer 클래스에 추가
CognitaRiskAnalyzer.generate_risk_report = generate_risk_report
CognitaRiskAnalyzer._generate_recommendations = _generate_recommendations
CognitaRiskAnalyzer.batch_analyze_department = batch_analyze_department_optimized

print("✅ 누락된 메서드들 추가 완료:")
print("   - generate_risk_report")
print("   - _generate_recommendations") 
print("   - batch_analyze_department")

✅ 누락된 메서드들 추가 완료:
   - generate_risk_report
   - _generate_recommendations
   - batch_analyze_department


셀 9: 데모 실행 및 결과 출력

In [16]:
# ------------------------------------------------------
# 셀 9: Cognita 관계형 위험도 분석 데모 실행
# ------------------------------------------------------

def run_cognita_analysis_demo(neo4j_manager):
    """Cognita 분석 시스템 데모 실행"""
    
    print("=" * 60)
    print("🎯 Cognita 관계형 위험도 분석 시작")
    print("=" * 60)
    
    analyzer = CognitaRiskAnalyzer(neo4j_manager)
    
    # 1. 샘플 직원 개별 분석
    print("\n📊 1. 개별 직원 위험도 분석:")
    print("-" * 40)
    
    # 샘플 직원 ID 가져오기
    query = """
    MATCH (e:Employee) 
    RETURN e.employee_id as id, e.name as name, e.department as dept
    ORDER BY e.employee_id 
    LIMIT 5
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        sample_employees = [(record['id'], record['name'], record['dept']) 
                          for record in result]
    
    if sample_employees:
        for emp_id, emp_name, dept in sample_employees[:3]:
            try:
                print(f"\n🔍 분석 대상: {emp_name} ({emp_id}) - {dept}")
                risk_metrics = analyzer.analyze_employee_risk(emp_id)
                
                print(f"  📈 전체 위험도: {risk_metrics.overall_risk_score:.3f} ({risk_metrics.risk_category})")
                print(f"  👥 사회적 고립: {risk_metrics.social_isolation_index:.3f}")
                print(f"  🕸️  네트워크 중심성: {risk_metrics.network_centrality_score:.3f}")
                print(f"  👨‍💼 관리자 불안정성: {risk_metrics.manager_instability_score:.3f}")
                print(f"  🔄 팀 변동성: {risk_metrics.team_volatility_index:.3f}")
                
                if risk_metrics.risk_factors:
                    print(f"  ⚠️  주요 위험 요인: {', '.join(risk_metrics.risk_factors)}")
                else:
                    print(f"  ✅ 주요 위험 요인: 없음")
                
                # 네트워크 상세 정보
                if risk_metrics.network_stats:
                    connections = risk_metrics.network_stats.get('direct_connections', 0)
                    strength = risk_metrics.network_stats.get('total_collaboration_strength', 0)
                    print(f"  🔗 직접 연결: {connections}명, 총 협업 강도: {strength:.2f}")
                
            except Exception as e:
                print(f"  ❌ 분석 실패: {str(e)}")
    
    # 2. 부서별 일괄 분석
    print(f"\n\n🏢 2. 부서별 위험도 분석:")
    print("-" * 40)
    
    # 부서 목록 조회 (직원 수 포함)
    query = """
    MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department)
    WITH dept.name as dept_name, count(emp) as emp_count
    WHERE emp_count >= 10
    RETURN dept_name, emp_count
    ORDER BY emp_count DESC
    LIMIT 2
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        departments = [(record['dept_name'], record['emp_count']) for record in result]
    
    for dept_name, emp_count in departments:
        try:
            print(f"\n🏛️  부서: {dept_name} ({emp_count}명)")
            
            # 샘플링으로 분석 시간 단축 (대용량 부서의 경우)
            if emp_count > 50:
                print(f"   📝 대용량 부서로 인해 샘플 분석 실시 (처음 20명)")
                sample_query = """
                MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department {name: $dept_name})
                RETURN emp.employee_id as id
                ORDER BY emp.employee_id
                LIMIT 20
                """
                
                with neo4j_manager.driver.session() as session:
                    sample_result = session.run(sample_query, dept_name=dept_name)
                    sample_ids = [record['id'] for record in sample_result]
                
                # 샘플 직원들 개별 분석
                dept_analysis = []
                for emp_id in sample_ids:
                    try:
                        risk_metrics = analyzer.analyze_employee_risk(emp_id)
                        dept_analysis.append(risk_metrics)
                    except Exception as e:
                        continue
            else:
                # 전체 부서 분석
                dept_analysis = analyzer.batch_analyze_department(dept_name)
            
            if dept_analysis:
                report = analyzer.generate_risk_report(dept_analysis)
                
                print(f"   📊 분석 결과:")
                print(f"     • 분석 인원: {report['분석_개요']['총_분석_인원']}명")
                print(f"     • 위험 분포: HIGH({report['위험_분포']['HIGH']}) / MEDIUM({report['위험_분포']['MEDIUM']}) / LOW({report['위험_분포']['LOW']})")
                print(f"     • 고위험 비율: {report['분석_개요']['고위험_비율']}")
                print(f"     • 평균 위험도: {report['평균_위험_점수']['overall_risk']:.3f}")
                
                # 주요 위험 요인
                top_factors = list(report['주요_위험_요인_빈도'].items())[:3]
                if top_factors:
                    print(f"     • 주요 위험 요인: {', '.join([f'{factor}({count}명)' for factor, count in top_factors])}")
                
                # 고위험 직원 (상위 3명)
                if report['고위험_직원_상세']:
                    print(f"     • 고위험 직원:")
                    for emp in report['고위험_직원_상세'][:3]:
                        print(f"       - {emp['employee_id']}: {emp['overall_risk_score']:.3f}")
                
                # 권장 조치사항
                print(f"     • 권장 조치:")
                for i, recommendation in enumerate(report['권장_조치사항'][:2], 1):
                    print(f"       {i}. {recommendation}")
            else:
                print(f"   ❌ 분석 데이터 없음")
                
        except Exception as e:
            print(f"   ❌ 부서 분석 실패: {str(e)}")
    
    print(f"\n" + "=" * 60)
    print(f"✅ Cognita 관계형 위험도 분석 완료")
    print(f"=" * 60)

# 실행 준비 완료 메시지
print("✅ Cognita 관계형 위험도 분석 시스템 준비 완료")
print("\n사용법:")
print("  run_cognita_analysis_demo(neo4j_manager)")
print("\n또는 개별 분석:")
print("  analyzer = CognitaRiskAnalyzer(neo4j_manager)")
print("  risk_metrics = analyzer.analyze_employee_risk('EMPLOYEE_ID')")

✅ Cognita 관계형 위험도 분석 시스템 준비 완료

사용법:
  run_cognita_analysis_demo(neo4j_manager)

또는 개별 분석:
  analyzer = CognitaRiskAnalyzer(neo4j_manager)
  risk_metrics = analyzer.analyze_employee_risk('EMPLOYEE_ID')


셀 10: Neo4j 연결 설정

In [17]:
# ------------------------------------------------------
# 셀 10: Neo4j 연결 설정 (Cognita 분석용)
# ------------------------------------------------------

import xml.etree.ElementTree as ET
import pandas as pd
import json
from neo4j import GraphDatabase
from pathlib import Path
import logging
from datetime import datetime
import time

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Neo4j 연결 설정 (이전에 성공한 설정 사용)
NEO4J_CONFIG = {
    "uri": "bolt://54.162.43.24:7687",
    "username": "neo4j",
    "password": "resident-success-moss"
}

class Neo4jManager:
    """Neo4j 데이터베이스 연결 및 관리 클래스"""
    
    def __init__(self, uri, username, password, max_retries=3):
        self.uri = uri
        self.username = username
        self.password = password
        self.max_retries = max_retries
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Neo4j 연결 설정"""
        for attempt in range(self.max_retries):
            try:
                logger.info(f"Neo4j 연결 시도 {attempt + 1}/{self.max_retries}: {self.uri}")
                
                self.driver = GraphDatabase.driver(
                    self.uri, 
                    auth=(self.username, self.password),
                    max_connection_lifetime=30,
                    max_connection_pool_size=10,
                    connection_acquisition_timeout=30,
                    connection_timeout=10
                )
                
                # 연결 테스트
                with self.driver.session() as session:
                    result = session.run("RETURN 'Connected' as status")
                    logger.info("Neo4j 연결 성공")
                    return
                    
            except Exception as e:
                logger.warning(f"연결 시도 {attempt + 1} 실패: {str(e)}")
                
                if self.driver:
                    try:
                        self.driver.close()
                    except:
                        pass
                    self.driver = None
                
                if attempt < self.max_retries - 1:
                    wait_time = (attempt + 1) * 2
                    logger.info(f"{wait_time}초 후 재시도...")
                    time.sleep(wait_time)
                else:
                    raise ConnectionError(f"Neo4j 연결 실패: {str(e)}")
    
    def close(self):
        if self.driver:
            try:
                self.driver.close()
                logger.info("Neo4j 연결 종료")
            except Exception as e:
                logger.warning(f"연결 종료 중 오류: {str(e)}")
        self.driver = None

# Neo4j 연결 생성
try:
    print("Neo4j 연결 설정 중...")
    neo4j_manager = Neo4jManager(
        NEO4J_CONFIG['uri'], 
        NEO4J_CONFIG['username'], 
        NEO4J_CONFIG['password']
    )
    
    # 연결 확인 및 기본 통계
    with neo4j_manager.driver.session() as session:
        # 데이터 존재 확인
        result = session.run("MATCH (e:Employee) RETURN count(e) as employee_count")
        employee_count = result.single()["employee_count"]
        
        result = session.run("MATCH ()-[r:COLLABORATES_WITH]->() RETURN count(r) as collab_count")
        collab_count = result.single()["collab_count"]
        
        print(f"연결 성공! 데이터 확인:")
        print(f"  - 직원 수: {employee_count:,}명")
        print(f"  - 협업 관계 수: {collab_count:,}개")
        
        if employee_count == 0:
            print("경고: 직원 데이터가 없습니다. 먼저 그래프를 생성하세요.")
        
except Exception as e:
    print(f"Neo4j 연결 실패: {str(e)}")
    print("해결 방법:")
    print("1. Neo4j 서버가 실행 중인지 확인")
    print("2. 연결 정보 확인")
    print("3. 네트워크 연결 상태 확인")
    neo4j_manager = None

INFO:__main__:Neo4j 연결 시도 1/3: bolt://54.162.43.24:7687


Neo4j 연결 설정 중...


INFO:__main__:Neo4j 연결 성공


연결 성공! 데이터 확인:
  - 직원 수: 1,470명
  - 협업 관계 수: 2,186,936개


셀 11: 샘플 직원 ID 조회

In [18]:
# ------------------------------------------------------
# 셀 11: 테스트용 샘플 직원 ID 조회
# ------------------------------------------------------

def get_sample_employee_ids(neo4j_manager, limit=10):
    """테스트용 샘플 직원 ID 조회"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 없습니다.")
        return []
    
    query = """
    MATCH (e:Employee)
    RETURN e.employee_id as id, 
           e.name as name, 
           e.department as department,
           e.job_role as job_role,
           e.risk_tier as risk_tier
    ORDER BY e.employee_id
    LIMIT $limit
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query, limit=limit)
            employees = []
            
            for record in result:
                employees.append({
                    'id': record['id'],
                    'name': record['name'],
                    'department': record['department'],
                    'job_role': record['job_role'],
                    'risk_tier': record['risk_tier']
                })
            
            return employees
            
    except Exception as e:
        print(f"직원 조회 실패: {str(e)}")
        return []

# 샘플 직원들 조회
if neo4j_manager:
    sample_employees = get_sample_employee_ids(neo4j_manager, 15)
    
    if sample_employees:
        print("테스트 가능한 직원 목록:")
        print("-" * 80)
        for i, emp in enumerate(sample_employees):
            print(f"{i+1:2d}. {emp['id']:<12} | {emp['name']:<15} | {emp['department']:<20} | {emp['job_role']:<20} | {emp['risk_tier']}")
        
        print(f"\n첫 번째 직원으로 테스트: {sample_employees[0]['id']}")
        test_employee_id = sample_employees[0]['id']
    else:
        print("조회 가능한 직원이 없습니다.")
        test_employee_id = None
else:
    print("Neo4j 연결을 먼저 설정하세요.")
    test_employee_id = None

테스트 가능한 직원 목록:
--------------------------------------------------------------------------------
 1. 1            | Employee_1      | Sales                | Sales Executive      | High
 2. 10           | Employee_10     | Research & Development | Laboratory Technician | Neutral
 3. 100          | Employee_100    | Sales                | Sales Executive      | Neutral
 4. 1001         | Employee_1001   | Research & Development | Laboratory Technician | High
 5. 1002         | Employee_1002   | Research & Development | Laboratory Technician | High
 6. 1003         | Employee_1003   | Sales                | Sales Executive      | High
 7. 1004         | Employee_1004   | Research & Development | Research Scientist   | High
 8. 1005         | Employee_1005   | Research & Development | Manufacturing Director | High
 9. 1006         | Employee_1006   | Research & Development | Research Scientist   | High
10. 1007         | Employee_1007   | Research & Development | Manufacturing Director | Hi

셀 12: Cognita 분석기 테스트 실행

In [19]:
# ------------------------------------------------------
# 셀 12: Cognita 분석기 개별 직원 테스트
# ------------------------------------------------------

def test_individual_analysis(neo4j_manager, employee_id):
    """개별 직원 분석 테스트"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 필요합니다.")
        return None
    
    if not employee_id:
        print("유효한 직원 ID가 필요합니다.")
        return None
    
    try:
        print(f"분석 시작: {employee_id}")
        print("-" * 50)
        
        # Cognita 분석기 초기화
        analyzer = CognitaRiskAnalyzer(neo4j_manager)
        
        # 위험도 분석 실행
        risk_metrics = analyzer.analyze_employee_risk(employee_id)
        
        # 결과 출력
        print("분석 결과:")
        print(f"  직원 ID: {risk_metrics.employee_id}")
        print(f"  전체 위험도: {risk_metrics.overall_risk_score:.3f} ({risk_metrics.risk_category})")
        print()
        print("세부 지표:")
        print(f"  사회적 고립 지수: {risk_metrics.social_isolation_index:.3f}")
        print(f"  네트워크 중심성: {risk_metrics.network_centrality_score:.3f}")
        print(f"  관리자 불안정성: {risk_metrics.manager_instability_score:.3f}")
        print(f"  팀 변동성: {risk_metrics.team_volatility_index:.3f}")
        print()
        
        if risk_metrics.risk_factors:
            print(f"주요 위험 요인: {', '.join(risk_metrics.risk_factors)}")
        else:
            print("주요 위험 요인: 없음")
        
        # 네트워크 상세 정보
        if risk_metrics.network_stats:
            print()
            print("네트워크 상세 정보:")
            for key, value in risk_metrics.network_stats.items():
                if isinstance(value, float):
                    print(f"  {key}: {value:.3f}")
                else:
                    print(f"  {key}: {value}")
        
        print()
        print("=" * 50)
        
        return risk_metrics
        
    except Exception as e:
        print(f"분석 실패: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 테스트 실행
if neo4j_manager and test_employee_id:
    print("개별 직원 분석 테스트 실행")
    print("=" * 50)
    
    result = test_individual_analysis(neo4j_manager, test_employee_id)
    
    if result:
        print("테스트 성공!")
    else:
        print("테스트 실패")
else:
    print("Neo4j 연결 또는 테스트 직원 ID가 없습니다.")

INFO:__main__:직원 1 위험도 분석 시작


개별 직원 분석 테스트 실행
분석 시작: 1
--------------------------------------------------
분석 결과:
  직원 ID: 1
  전체 위험도: 0.530 (MEDIUM)

세부 지표:
  사회적 고립 지수: 0.600
  네트워크 중심성: 0.700
  관리자 불안정성: 0.800
  팀 변동성: 0.300

주요 위험 요인: 관리자_불안정성

네트워크 상세 정보:
  degree_centrality: 1.000
  strength_centrality: 1.000
  project_centrality: 0.000
  quality_score: 0.800
  direct_connections: 25
  avg_strength: 1.000

테스트 성공!


셀 14: 부서별 분석 테스트

In [20]:
# ------------------------------------------------------
# 셀 14: 부서별 위험도 분석 테스트 (샘플링)
# ------------------------------------------------------

def test_department_analysis(neo4j_manager, max_employees_per_dept=10):
    """부서별 위험도 분석 테스트 (샘플링)"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 필요합니다.")
        return
    
    # 직원 수가 많은 부서 조회
    query = """
    MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department)
    WITH dept.name as dept_name, count(emp) as emp_count
    WHERE emp_count >= 5
    RETURN dept_name, emp_count
    ORDER BY emp_count DESC
    LIMIT 2
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query)
            departments = [(record['dept_name'], record['emp_count']) for record in result]
        
        if not departments:
            print("분석할 부서가 없습니다.")
            return
        
        analyzer = CognitaRiskAnalyzer(neo4j_manager)
        
        for dept_name, total_count in departments:
            print(f"\n부서 분석: {dept_name} (전체 {total_count}명)")
            print("=" * 60)
            
            # 샘플 직원 조회 (분석 시간 단축)
            sample_query = """
            MATCH (emp:Employee)-[:WORKS_IN]->(dept:Department {name: $dept_name})
            RETURN emp.employee_id as id, emp.name as name
            ORDER BY emp.employee_id
            LIMIT $limit
            """
            
            with neo4j_manager.driver.session() as session:
                sample_result = session.run(sample_query, 
                                           dept_name=dept_name, 
                                           limit=max_employees_per_dept)
                sample_employees = [(record['id'], record['name']) for record in sample_result]
            
            if not sample_employees:
                print("샘플 직원이 없습니다.")
                continue
            
            print(f"샘플 분석: {len(sample_employees)}명 (전체 {total_count}명 중)")
            
            dept_results = []
            for i, (emp_id, emp_name) in enumerate(sample_employees):
                try:
                    risk_metrics = analyzer.analyze_employee_risk(emp_id)
                    dept_results.append(risk_metrics)
                    
                    if (i + 1) % 3 == 0:  # 3명마다 진행 상황 표시
                        print(f"  진행: {i + 1}/{len(sample_employees)}")
                        
                except Exception as e:
                    print(f"  {emp_id} 분석 실패: {str(e)}")
                    continue
            
            if dept_results:
                # 부서 리포트 생성
                report = analyzer.generate_risk_report(dept_results)
                
                print(f"\n{dept_name} 부서 분석 결과:")
                print("-" * 40)
                print(f"  분석 인원: {len(dept_results)}명")
                print(f"  위험 분포: HIGH({report['위험_분포']['HIGH']}) / MEDIUM({report['위험_분포']['MEDIUM']}) / LOW({report['위험_분포']['LOW']})")
                print(f"  고위험 비율: {report['분석_개요']['고위험_비율']}")
                print(f"  평균 위험도: {report['평균_위험_점수']['overall_risk']:.3f}")
                
                # 고위험 직원
                if report['고위험_직원_상세']:
                    print(f"  고위험 직원:")
                    for emp in report['고위험_직원_상세'][:3]:
                        print(f"    - {emp['employee_id']}: {emp['overall_risk_score']:.3f}")
                
                # 주요 위험 요인
                top_factors = list(report['주요_위험_요인_빈도'].items())[:3]
                if top_factors:
                    print(f"  주요 위험 요인:")
                    for factor, count in top_factors:
                        print(f"    - {factor}: {count}명")
                
                # 권장 조치사항
                print(f"  권장 조치:")
                for i, rec in enumerate(report['권장_조치사항'][:2], 1):
                    print(f"    {i}. {rec}")
            
    except Exception as e:
        print(f"부서 분석 실패: {str(e)}")
        import traceback
        traceback.print_exc()

# 부서별 분석 테스트 실행
if neo4j_manager:
    test_department_analysis(neo4j_manager)
else:
    print("Neo4j 연결을 먼저 설정하세요.")

INFO:__main__:직원 10 위험도 분석 시작



부서 분석: Research & Development (전체 961명)
샘플 분석: 10명 (전체 961명 중)


INFO:__main__:직원 1001 위험도 분석 시작
INFO:__main__:직원 1002 위험도 분석 시작
INFO:__main__:직원 1004 위험도 분석 시작


  진행: 3/10


INFO:__main__:직원 1005 위험도 분석 시작
INFO:__main__:직원 1006 위험도 분석 시작
INFO:__main__:직원 1007 위험도 분석 시작


...

  진행: 3/10


INFO:__main__:직원 1036 위험도 분석 시작
INFO:__main__:직원 1037 위험도 분석 시작
INFO:__main__:직원 1038 위험도 분석 시작


  진행: 6/10


INFO:__main__:직원 1039 위험도 분석 시작
INFO:__main__:직원 1040 위험도 분석 시작
INFO:__main__:직원 1044 위험도 분석 시작


  진행: 9/10

Sales 부서 분석 결과:
----------------------------------------
  분석 인원: 10명
  위험 분포: HIGH(0) / MEDIUM(8) / LOW(2)
  고위험 비율: 0.0%
  평균 위험도: 0.470
  주요 위험 요인:
    - 관리자_불안정성: 9명
  권장 조치:
    1. 관리자 교육 및 코칭 강화
    2. 관리 스팬 최적화 검토


In [21]:
# ------------------------------------------------------
# 셀 15: 최적화된 성능 테스트
# ------------------------------------------------------

import time

def performance_test(neo4j_manager):
    """최적화 전후 성능 비교 테스트"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 필요합니다.")
        return
    
    # 테스트용 직원 ID 가져오기
    query = "MATCH (e:Employee) RETURN e.employee_id as id LIMIT 5"
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        test_employees = [record['id'] for record in result]
    
    if not test_employees:
        print("테스트할 직원이 없습니다.")
        return
    
    analyzer = CognitaRiskAnalyzer(neo4j_manager)
    
    print("성능 테스트 시작...")
    print("=" * 50)
    
    total_start_time = time.time()
    results = []
    
    for i, emp_id in enumerate(test_employees):
        print(f"\n{i+1}. 직원 {emp_id} 분석:")
        
        start_time = time.time()
        
        try:
            risk_metrics = analyzer.analyze_employee_risk(emp_id)
            end_time = time.time()
            
            elapsed = end_time - start_time
            results.append({
                'employee_id': emp_id,
                'elapsed_time': elapsed,
                'risk_score': risk_metrics.overall_risk_score,
                'success': True
            })
            
            print(f"   완료: {elapsed:.2f}초")
            print(f"   위험도: {risk_metrics.overall_risk_score:.3f} ({risk_metrics.risk_category})")
            print(f"   주요 요인: {', '.join(risk_metrics.risk_factors[:2])}")
            
        except Exception as e:
            end_time = time.time()
            elapsed = end_time - start_time
            
            results.append({
                'employee_id': emp_id,
                'elapsed_time': elapsed,
                'error': str(e),
                'success': False
            })
            
            print(f"   실패: {elapsed:.2f}초 - {str(e)}")
    
    total_end_time = time.time()
    total_elapsed = total_end_time - total_start_time
    
    # 성능 통계
    print(f"\n" + "=" * 50)
    print("성능 테스트 결과:")
    print("=" * 50)
    
    successful_results = [r for r in results if r['success']]
    
    if successful_results:
        avg_time = sum(r['elapsed_time'] for r in successful_results) / len(successful_results)
        min_time = min(r['elapsed_time'] for r in successful_results)
        max_time = max(r['elapsed_time'] for r in successful_results)
        
        print(f"성공률: {len(successful_results)}/{len(results)} ({len(successful_results)/len(results)*100:.1f}%)")
        print(f"총 소요 시간: {total_elapsed:.2f}초")
        print(f"평균 분석 시간: {avg_time:.2f}초/명")
        print(f"최소 분석 시간: {min_time:.2f}초")
        print(f"최대 분석 시간: {max_time:.2f}초")
        
        # 성능 등급 평가
        if avg_time < 2.0:
            performance_grade = "우수 (2초 미만)"
        elif avg_time < 5.0:
            performance_grade = "양호 (5초 미만)"
        elif avg_time < 10.0:
            performance_grade = "보통 (10초 미만)"
        else:
            performance_grade = "개선 필요 (10초 이상)"
        
        print(f"성능 등급: {performance_grade}")
        
        # 처리량 계산
        throughput = len(successful_results) / total_elapsed * 3600  # 시간당 처리량
        print(f"예상 처리량: {throughput:.0f}명/시간")
        
        # 전체 직원 처리 시간 예측
        total_employees = 1470  # 전체 직원 수
        estimated_total_time = total_employees * avg_time / 60  # 분 단위
        print(f"전체 직원 분석 예상 시간: {estimated_total_time:.1f}분")
        
    else:
        print("모든 테스트 실패")
    
    return results

# 성능 테스트 실행
if neo4j_manager:
    performance_results = performance_test(neo4j_manager)
else:
    print("Neo4j 연결을 먼저 설정하세요.")

INFO:__main__:직원 1 위험도 분석 시작


성능 테스트 시작...

1. 직원 1 분석:


INFO:__main__:직원 2 위험도 분석 시작


   완료: 0.85초
   위험도: 0.530 (MEDIUM)
   주요 요인: 관리자_불안정성

2. 직원 2 분석:


INFO:__main__:직원 4 위험도 분석 시작


   완료: 0.87초
   위험도: 0.530 (MEDIUM)
   주요 요인: 관리자_불안정성

3. 직원 4 분석:


INFO:__main__:직원 5 위험도 분석 시작


   완료: 1.76초
   위험도: 0.530 (MEDIUM)
   주요 요인: 관리자_불안정성

4. 직원 5 분석:


INFO:__main__:직원 7 위험도 분석 시작


   완료: 0.91초
   위험도: 0.530 (MEDIUM)
   주요 요인: 관리자_불안정성

5. 직원 7 분석:
   완료: 0.90초
   위험도: 0.530 (MEDIUM)
   주요 요인: 관리자_불안정성

성능 테스트 결과:
성공률: 5/5 (100.0%)
총 소요 시간: 5.29초
평균 분석 시간: 1.06초/명
최소 분석 시간: 0.85초
최대 분석 시간: 1.76초
성능 등급: 우수 (2초 미만)
예상 처리량: 3403명/시간
전체 직원 분석 예상 시간: 25.9분
